## Stuff to load everything

In [1]:
import rootutils

rootutils.setup_root("/TopoBenchmarkX", indicator=".project-root", pythonpath=True)

import hydra
from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/usr/local/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [2]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader

data_loader = HypergraphLoader(config)
data = data_loader.load()

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072


In [3]:
print(data)

Data(x=[2708, 1433], edge_index=[2708, 4585], y=[2708], n_x=2708, num_hyperedges=1072, num_class=7)


In [5]:
import torch_geometric
import torch

In [6]:
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.utils import to_undirected

In [7]:
edges = data['edge_index'].coalesce().indices()
edges = to_undirected(edges)

In [8]:
subset, _, _, _ = k_hop_subgraph(2, 1, edges)
print(subset)

tensor([   2, 3107, 3655])


In [9]:
class KHopLifting(torch_geometric.transforms.BaseTransform):
    def __init__(self, k=1):
        super().__init__()
        self.k = k
        
    def forward(self, data: torch_geometric.data.Data) -> dict:
        for i, d in enumerate(data):
            n_nodes = d.x.shape[0]
            incidence_1 = torch.zeros(n_nodes, n_nodes)
            edge_index = torch_geometric.utils.to_undirected(d.edge_index)
            for n in range(n_nodes):
                neighbors, _, _, _ = k_hop_subgraph(n, self.k, edge_index)
                incidence_1[n, neighbors] = 1
            incidence_1 = torch.Tensor(incidence_1).to_sparse_coo()
            data[i]["hyperedges"] = incidence_1

        return data

In [10]:
data_mutag = torch_geometric.datasets.TUDataset(root="../data/graph",name="MUTAG", transform=KHopLifting(), force_reload=True)

Processing...
Done!


In [11]:
print(data_mutag)

MUTAG(188)


In [12]:
for d in data_mutag:
    print(d)
    print(d.edge_index)
    # print(torch_geometric.utils.to_dense_adj(d.hyperedges.transpose(1,0))[0].to_sparse_coo())
    break

AttributeError: 'tuple' object has no attribute 'x'

In [14]:
model = hydra.utils.instantiate(config.model)

/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout_workaround' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout_workaround'])`.
/usr/local/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'readout' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['readout'])`.


In [15]:
# This is ugly (also not working but because the model is for cora)

for d in data_mutag:
    r = model(d.x, torch_geometric.utils.to_dense_adj(d.hyperedges.transpose(1,0))[0].to_sparse_coo())
    print(r)
    break

NotImplementedError: Could not run 'aten::max' with arguments from the 'SparseCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::max' is only available for these backends: [CPU, CUDA, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMeta, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, Python

In [16]:
def get_cora(cfg):
    data_dir = cfg["data_dir"]
    print(f"Loading {cfg['data_domain']} dataset name: {cfg['data_name']}")
    
    data = torch_geometric.datasets.Planetoid(data_dir, "Cora")
    return data

In [20]:
from torch_geometric.loader.dataloader import DataLoader

data = get_cora(config.data)

Loading hypergraph dataset name: cora


Processing...
Done!


In [52]:
lift = KHopLifting(1)

data = lift(data)

for d in data:
    print(d)


KeyError: 'hyperedges'

In [44]:
print(data[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
